In [1]:
!pip install google-cloud-aiplatform --upgrade --user --quiet

In [2]:
import pandas as pd
import seaborn as sns
from IPython.display import Markdown, display
from vertexai.language_models import TextGenerationModel

2023-09-13 18:53:12.961748: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 18:53:14.025208: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-09-13 18:53:14.025347: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [3]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

In [4]:
df = pd.read_json("resources/datasets/val_sentence_pairs.json")
df.head()

,article_id,article_sent,correct_sent,correct_sent_id,incorrect_sent,incorrect_sent_id
0,67c3b0a2da09ce32a83c098da1e0aff034e790d0,the abc have reported that those who receive c...,those who receive centrelink payments made up ...,84-0,the abc have reported that those who receive c...,40-0
1,7c4ae3beb354e0a9f11fa09f8fbb6613a50db5cc,five ambitious clubs are locked in a scramble ...,five ambitious clubs are locked in a scramble ...,44-1,five ambitious clubs are locked in a bid for t...,17-1
2,7e37b3e3623ae0ab6568e0e73584b16fae0cc00e,but it wasn't until last year that the 25 year...,the 25 year old from pennsylvania went on a le...,45-1,the 25 year old from pennsylvania went viral o...,4-1
3,cdbfba47929b459daf474a1a3dee37d2351729ea,seven games involving nimes were investigated ...,seven games involving nimes were investigated ...,93-2,seven games involving nimes were arrested last...,83-2
4,696ac804af5fcab8853c583f5c97e1eda6e2493a,the driver's side of the windscreen immediatel...,driver's side of the windscreen immediately sh...,61-2,driver's side were immediately shatters and fa...,42-2


In [6]:
from prompt_mgr import PromptMgr

pm = PromptMgr(src_dir = 'resources/environments/answer_first/prompts')

In [10]:
def query_model(row, model_predictor, prompt_mgr, swap_answers = False):
    if swap_answers: 
        prompt = prompt_mgr.bind('consistent').render(
                    article_sent=row['article_sent'], 
                    option_a=row['incorrect_sent'],
                    option_b=row['correct_sent'])
    else: 
        prompt = prompt_mgr.bind('consistent').render(
                    article_sent=row['article_sent'], 
                    option_a=row['correct_sent'],
                    option_b=row['incorrect_sent'])
    system_prompt = prompt_mgr.bind('system').render()
    prompt_data = f"{system_prompt}\n{prompt}"
    
    response = model_predictor.predict(
        prompt=prompt_data
    )
    
    return {'output': response.text} 

In [8]:
result = query_model(df.loc[0], generation_model, pm, swap_answers=True)

prompt_data: You are an assistant that helps determine the factuality of statements. Being precise, factual and following the user's instructions is important for this task. 
It is acceptable to answer in incomplete sentences e.g. just say 'x' and not 'The correct answer is x'. 

Decide which of the following summary is more consistent with the article
sentence. 

Note that consistency means
all information in the summary is supported by the article.

Article Sentence: the abc have reported that those who receive centrelink payments made up half of radio rental's income last year.
Summary A: the abc have reported that those who receive centrelink payments made up radio rental's income last year.
Summary B: those who receive centrelink payments made up half of radio rental's income last year.

Answer (A or B):



In [9]:
print(result['output'])

B


In [ ]:
import time

# Create an empty list to store the results
results_list = []

# Iterate over each row in the DataFrame. sleep 1 min every 60 requests
for i in range(len(df)):
    result = query_model(df.loc[i], generation_model, pm, swap_answers=True)
    results_list.append(result)
    if i % 60 == 0:
        time.sleep(60)

# Create a new DataFrame from the results list
results_df = pd.DataFrame(results_list)

In [18]:
len(results_list)

373

In [15]:
len(df)

373

In [19]:
results_df.to_json('PaLM2-text-bison@001-output-swapped.json')

In [20]:
results_df

,output
0,B
1,B
2,B
3,B
4,B
...,...
368,B
369,A
370,A
371,B


In [25]:
# Create an empty list to store the results
results_list_no_swap = []

# Iterate over each row in the DataFrame
for i in range(len(df)):
    result = query_model(df.loc[i], generation_model, pm, swap_answers=False)
    results_list_no_swap.append(result)
    if i % 60 == 0:
        time.sleep(60)

# Create a new DataFrame from the results list
results_df_no_swap = pd.DataFrame(results_list_no_swap)

In [26]:
len(results_df_no_swap)

373

In [27]:
results_df_no_swap.to_json('PaLM2-text-bison@001-output-no-swapped.json')

In [ ]:
results_df.rename(columns={'output': 'palm_swapped_output'}, inplace=True)

In [28]:
results_df_no_swap.rename(columns={'output': 'palm_no_swap_output'}, inplace=True)
input_df = df

combined_df = pd.concat([input_df, 
                         results_df, 
                         results_df_no_swap,], axis=1)
combined_df.head()

,article_id,article_sent,correct_sent,correct_sent_id,incorrect_sent,incorrect_sent_id,palm_swapped_output,palm_no_swap_output
0,67c3b0a2da09ce32a83c098da1e0aff034e790d0,the abc have reported that those who receive c...,those who receive centrelink payments made up ...,84-0,the abc have reported that those who receive c...,40-0,B,A
1,7c4ae3beb354e0a9f11fa09f8fbb6613a50db5cc,five ambitious clubs are locked in a scramble ...,five ambitious clubs are locked in a scramble ...,44-1,five ambitious clubs are locked in a bid for t...,17-1,B,A
2,7e37b3e3623ae0ab6568e0e73584b16fae0cc00e,but it wasn't until last year that the 25 year...,the 25 year old from pennsylvania went on a le...,45-1,the 25 year old from pennsylvania went viral o...,4-1,B,A
3,cdbfba47929b459daf474a1a3dee37d2351729ea,seven games involving nimes were investigated ...,seven games involving nimes were investigated ...,93-2,seven games involving nimes were arrested last...,83-2,B,A
4,696ac804af5fcab8853c583f5c97e1eda6e2493a,the driver's side of the windscreen immediatel...,driver's side of the windscreen immediately sh...,61-2,driver's side were immediately shatters and fa...,42-2,B,A


In [22]:
def is_correct(row): 
    if (row[0] == 'B' and row[1] == 'A'):
        return 'Y'
    if (row[0] == 'A' and row[1] == 'A'):
        return 'AA'
    if (row[0] == 'B' and row[1] == 'B'):
        return 'BB'
    if (row[0] == 'A' and row[1] == 'B'):
        return 'N'
    return 'No answer'

In [29]:
combined_df['palm_analysis_result'] = combined_df[['palm_swapped_output', 'palm_no_swap_output']].apply(is_correct, axis=1)

In [30]:
def acc_bias(col):
    results = combined_df[col].value_counts()/len(combined_df)
    acc = results['Y']
    in_acc = results['N']
    no_answer = results['No answer']
    if 'AA' not in results:
        results['AA'] = 0
    if 'BB' not in results: 
        results['BB'] = 0
        
    bias = abs(results['AA'] - results['BB'])
    aa_ratio = results['AA'] * 100
    bb_ratio = results['BB'] * 100
    if results['AA'] > results['BB']:
        towards = 'A'
    else:
        towards = 'B'
        
    return acc, in_acc, no_answer, aa_ratio, bb_ratio, bias, towards

In [31]:
for m in ['palm',]:
    acc, in_acc, no_answer, aa_ratio, bb_ratio, bias, towards = acc_bias(m+'_analysis_result')
    acc = acc*100
    in_acc = in_acc*100
    no_answer = no_answer*100
    bias = bias*100
    print(f'{m}:   \tAccuracy: {acc:.1f}%     \tInaccuracy: {in_acc:.1f}%     \tNO_Answer: {no_answer:.1f}%     \tAA: {aa_ratio:.1f}%\tBB: {bb_ratio:.1f}%\tBias: {bias:.1f}% towards {towards}')

palm:   	Accuracy: 81.5%     	Inaccuracy: 2.9%     	NO_Answer: 2.7%     	AA: 0.5%	BB: 12.3%	Bias: 11.8% towards B
